# WoW Analytics - Data Feeder

This notebook fetches auction data from the Blizzard API and stores it as Parquet files.

## Architecture

This application follows hexagonal architecture:
- **Domain**: Pure business entities (Auction, Item, etc.)
- **Ports**: Abstract interfaces (BlizzardAPIPort, ParquetStoragePort)
- **Adapters**: Concrete implementations (BlizzardAPIClient, LocalParquetWriter, S3ParquetWriter)
- **Use Cases**: Business logic orchestration

In [8]:
##############################
# Configuration & base setup #
##############################

from config import Config
from adapters.blizzard_api import BlizzardAPIClient
from adapters.storage import LocalParquetWriter, S3ParquetWriter
from usecases import FetchAndStoreAuctionsUseCase

# Load configuration
CONFIG = Config.load()

print(f"Region: {CONFIG.blizzard.api_region.value}")
print(f"Storage type: {CONFIG.storage.type.value}")

if CONFIG.storage.type.value == "local":
    print(f"Storage directory: {CONFIG.storage.local.root_directory}")
else:
    print(f"S3 bucket: {CONFIG.storage.s3.bucket}")

Region: eu
Storage type: local
Storage directory: /Users/julien.leloup/Personal/ShadowDrive/wow-analytics-data


In [ ]:
##############################
# Create storage adapter     #
##############################

# Choose storage based on configuration
if CONFIG.storage.type.value == "local":
    storage = LocalParquetWriter(
        root_dir=CONFIG.storage.local.root_directory,
        default_compression=CONFIG.storage.local.compression,
        region=CONFIG.blizzard.api_region.value,
    )
else:
    storage = S3ParquetWriter(
        bucket=CONFIG.storage.s3.bucket,
        prefix=CONFIG.storage.s3.prefix,
        region=CONFIG.storage.s3.region,
        default_compression=CONFIG.storage.s3.compression,
        endpoint_url=CONFIG.storage.s3.endpoint_url,
        aws_access_key_id=CONFIG.storage.s3.aws_access_key_id,
        aws_secret_access_key=(
            CONFIG.storage.s3.aws_secret_access_key.get_secret_value()
            if CONFIG.storage.s3.aws_secret_access_key
            else None
        ),
        api_region=CONFIG.blizzard.api_region.value,
    )

print(f"Storage initialized at: {storage.base_path}")

In [10]:
##############################
# Fetch and store auction data
##############################

async def main():
    # Create the API client adapter (uses async context manager)
    async with BlizzardAPIClient(
        client_id=CONFIG.blizzard.api_client_id,
        client_secret=CONFIG.blizzard.api_client_secret.get_secret_value(),
        region=CONFIG.blizzard.api_region.value,
        locale=CONFIG.blizzard.api_locale,
    ) as api_client:
        # Create use case with injected dependencies
        use_case = FetchAndStoreAuctionsUseCase(
            api=api_client,
            storage=storage,
        )
        
        # Execute the use case - fetch and store realm metadata
        print("Fetching connected realms...")
        realms = await use_case.fetch_and_store_realm_metadata()
        print(f"Stored {len(realms)} realms")
        
        # Fetch and store auctions
        print("\nFetching realm auctions...")
        auction_paths = await use_case.fetch_and_store_realm_auctions()
        print(f"Stored auctions for {len(auction_paths)} realms")
        
        # Optionally fetch commodities
        print("\nFetching commodity auctions...")
        commodity_path = await use_case.fetch_and_store_commodities()
        if commodity_path:
            print(f"Stored commodities at: {commodity_path}")

# Run
await main()

Fetching connected realms...


AttributeError: 'str' object has no attribute 'get'